In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)

print(trainset.data.shape)

train_data_mean = train.data.mean( axis=(0,1,2) ) # 평균
train_data_std = train.data.std( axis=(0,1,2) ) # 표준편차


print(train_data_mean)
print(train_data_std)

#0~1 사이로 정규화 
train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 255

print(train_data_mean)
print(train_data_std)

Files already downloaded and verified
(50000, 32, 32, 3)
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [7]:
trans_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

trans_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])


trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=trans_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=0)

testset = datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=trans_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
len(trainloader)


40

In [9]:
len(testloader)


40

In [10]:
import torchvision.models.resnet as resnet

conv1x1=resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock= resnet.BasicBlock

In [11]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        
        self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=1)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
                    
                    

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [12]:
resnet50 = ResNet(resnet.Bottleneck, [3, 4, 6, 3], 10, True).to(device) 

In [13]:
resnet50

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [14]:

a=torch.Tensor(1,3,32,32).to(device)
out = resnet50(a)
print(out)

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [15]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(resnet50.parameters(), lr = 0.1, momentum = 0.9, weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [16]:
test_accuracy_list = []

def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, label in test_set:
            images = images.to(device)
            label = label.to(device)
            
            out = net(images)
            
            prediction = out.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
            
    acc = 100. * correct / len(testloader.dataset)
    test_accuracy_list.append(acc)        
    if save:
        torch.save(net.state_dict(), "./model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
        
    return acc

In [18]:
epochs = 50
train_loss_batch = []
train_loss_all = []
num_of_mini_batch = len(trainloader)

for epoch in range(epochs):
    avg_cost = 0 
    lr_sche.step()
    
    for idx, (img, label) in enumerate(trainloader):
        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        
        outs = resnet50(img)
        loss = criterion(outs, label)
        loss.backward()
        optimizer.step()
        
        train_loss_batch.append(loss)
        avg_cost = avg_cost + (loss.item()/num_of_mini_batch)
        
    print('Epoch:', '%02d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
          
print('Finished')

Epoch: 01 loss = 1.677011
Epoch: 02 loss = 1.521060
Epoch: 03 loss = 1.719975
Epoch: 04 loss = 1.651682
Epoch: 05 loss = 1.255224
Epoch: 06 loss = 1.573774
Epoch: 07 loss = 1.338568
Epoch: 08 loss = 1.682340
Epoch: 09 loss = 1.300557
Epoch: 10 loss = 1.038728
Epoch: 11 loss = 1.185616
Epoch: 12 loss = 1.347859
Epoch: 13 loss = 1.458608
Epoch: 14 loss = 1.074731
Epoch: 15 loss = 1.367572
Epoch: 16 loss = 1.013012
Epoch: 17 loss = 0.976214
Epoch: 18 loss = 1.160330
Epoch: 19 loss = 1.109112
Epoch: 20 loss = 0.850003
Epoch: 21 loss = 0.582815
Epoch: 22 loss = 0.518352
Epoch: 23 loss = 0.961250
Epoch: 24 loss = 0.864878
Epoch: 25 loss = 1.153503
Epoch: 26 loss = 0.679289
Epoch: 27 loss = 0.849255
Epoch: 28 loss = 0.912771
Epoch: 29 loss = 0.309781
Epoch: 30 loss = 0.698254
Epoch: 31 loss = 0.946739
Epoch: 32 loss = 1.218252
Epoch: 33 loss = 0.786644
Epoch: 34 loss = 1.079302
Epoch: 35 loss = 0.420097
Epoch: 36 loss = 0.519935
Epoch: 37 loss = 0.627564
Epoch: 38 loss = 0.364766
Epoch: 39 lo

In [19]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 78 %
